# Skupina B - Analýza výstavby dopravní infrastruktury v zemích Evropské Unie

Prvním krokem bude stažení potřebných dat. Níže jsou API odkazy na stažení dat.

In [ ]:
railway_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/rail_if_tracks$defaultview/1.0?compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'
highway_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/road_if_motorwa$defaultview/1.0?compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'
population_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/tps00001/1.0/*.*.*?c[freq]=A&c[indic_de]=JAN&c[geo]=EU27_2020,EA20,EA19,BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,FX,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,IS,LI,NO,CH,UK,BA,ME,MD,MK,GE,AL,RS,TR,UA,XK,AD,BY,MC,RU,SM,AM,AZ&c[TIME_PERIOD]=2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025&compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'

Definujme funkci pro jejich stažení.

In [ ]:
import requests
import io

def download_file(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Failed to retrieve data, code: {response.status_code}')
        return None

    file = io.BytesIO(response.content)

    return file

Nyní data stáhneme a otevřeme pomocí knihovny Pandas.

In [ ]:
railway_file = download_file(railway_url)
highway_file = download_file(highway_url)
population_file = download_file(population_url)

Pro ukázku budeme data načítat již stažená

In [ ]:
import pandas as pd

railway_data = pd.read_csv(railway_file)
highway_data = pd.read_csv(highway_file)
population_data = pd.read_csv(population_file)

Správnost načtení ověříme zobrazením hlaviček.

In [ ]:
railway_data.head()

In [ ]:
highway_data.head()

In [ ]:
population_data.head()

**Nyní můžeme začít data analyzovat.**
Začneme tím, že z dat získáme pouze evropské státy.
Vynecháme tedy souhrnná data.

In [ ]:
population_data['geo'].unique()

Vidíme, že souhrnná data jsou tam, kde v atributu *geo* je nějaká závorka. Vybereme tedy všechna kromě těchto dat. 

In [ ]:
highway_data = highway_data[~highway_data['geo'].str.contains(r'[()]', regex=True)]
railway_data = railway_data[~railway_data['geo'].str.contains(r'[()]', regex=True)]
population_data = population_data[~population_data['geo'].str.contains(r'[()]', regex=True)]

Tím jsme se zbavili souhrnných dat.

In [ ]:
population_data['geo'].unique()

Dále rozdělíme atribut geo na kód a název státu pro sjednocení s prostorovými daty.

In [ ]:
def split_geo_column(df, source_col='geo'):
    parts = df[source_col].str.split(':', n=1, expand=True)
    df['CNTR_CODE'] = parts.iloc[:, 0]
    df['COUNTRY_NAME'] = parts.iloc[:, 1]
    return df

In [ ]:
railway_data = split_geo_column(railway_data)
highway_data = split_geo_column(highway_data)
population_data = split_geo_column(population_data)

In [ ]:
population_data.head()

Nyní veškerá data spojíme s prostorovou složkou, aby bylo možné je zobrazit v mapě. Nejprve musíme však data načíst a přichystat.

In [ ]:
import geopandas as gpd
nuts_file = "NUTS_RG_10M_2024_3035.gpkg"
nuts_data = gpd.read_file(nuts_file)

A zobrazme opět hlavičku.

In [ ]:
nuts_data.head()

Data mají nyní atribut geometry, ten využijeme pro výpočet plochy. Data mají ale pro jeden stát více úrovní. Nás ale zajímají data pouze na úrovni státu, tedy hledáme, kde je LEVL_CODE = 0.

In [ ]:
countries = nuts_data[nuts_data["LEVL_CODE"] == 0]

Nyní máme data pro celé státy, abychom zobrazili data v mapě, připojíme k němu data o infrastruktuře a počtu obyvatel. Jelikož by kolidovaly názvy atributu OBS_VALUE, přejmenujeme OBS_VALUE podle vrstev. Musíme také myslet na sjednocení dat v letech, nalezneme tedy roky, pro které máme veškerá data. Nalezneme tedy minimum maxim a maximum minim a řádky s lety mimo tento rozsah vypustíme před přejmenováním sloupců.

In [ ]:
def available_years(df):
    return set(
        df.loc[df['OBS_VALUE'].notna(), 'TIME_PERIOD'].unique()
    )

common_years = (
    available_years(railway_data)
    & available_years(highway_data)
    & available_years(population_data)
)

In [ ]:
def drop_out_of_range(df, min_year, max_year, year_col='TIME_PERIOD'):
    return df[
        df[year_col].between(min_year, max_year)
    ]

min_year = min(common_years)
max_year = max(common_years)

railway_data = drop_out_of_range(railway_data, min_year, max_year)
highway_data = drop_out_of_range(highway_data, min_year, max_year)
population_data = drop_out_of_range(population_data, min_year, max_year)

print(railway_data["TIME_PERIOD"].unique())
print(highway_data["TIME_PERIOD"].unique())
print(population_data["TIME_PERIOD"].unique())

Nyní vyřešíme kolizi názvu sloupců a přichystáme přímo data pro sloučení, abychom neměli zbytečně moc sloupců. Musíme to dělat přímo po jednotlivých letech, proto budeme rovnou iterovat přes roky.

In [ ]:
for y in common_years:
    railway_y = railway_data[railway_data['TIME_PERIOD'] == y].groupby('CNTR_CODE')['OBS_VALUE'].first().reset_index().rename(columns={'OBS_VALUE': f'railway_{y}'})
    highway_y = highway_data[highway_data['TIME_PERIOD'] == y].groupby('CNTR_CODE')['OBS_VALUE'].first().reset_index().rename(columns={'OBS_VALUE': f'highway_{y}'})
    population_y = population_data[population_data['TIME_PERIOD'] == y].groupby('CNTR_CODE')['OBS_VALUE'].first().reset_index().rename(columns={'OBS_VALUE': f'population_{y}'})
    
    countries = countries.merge(railway_y, on='CNTR_CODE', how='left')
    countries = countries.merge(highway_y, on='CNTR_CODE', how='left')
    countries = countries.merge(population_y, on='CNTR_CODE', how='left')

Nyní máme přidány 3xpočet společných let nových sloupců s daty. Zobrazíme hlavičku.

In [ ]:
countries.head()

A můžeme vizualizovat v prostoru. Vytvoříme si funkci, která zobrazí pouze požadovaný sloupec pro lepší čitelnost.

In [ ]:
def explore_column(df, columns, legend=True, colors='OrRd'):
    df_copy = df[['NAME_LATN', 'geometry']].copy()
    for column in columns:
        df_copy[column] = df[column]

    # We excpect the first column to be the one to display legend for
    return df_copy.explore(columns[0], legend=legend, cmap=colors)

In [ ]:
#countries.explore('railway_2015', legend=True, cmap='OrRd')
explore_column(countries, ['railway_2015', 'population_2015'])

Toto ale nic neříká o tom, jak se státům daří, vypočtěme index nárustu délky infrastruktury dělený plochou a počtem obyvatel v každém roce.

In [ ]:
for y in common_years:
    countries[f"railway_pc_{y}"] = countries[f"railway_{y}"] / (countries[f"population_{y}"] / 1e6)   # PC - per capita
    countries[f"railway_pa_{y}"] = countries[f"railway_{y}"] / (countries.area / 1e6)         # PA - per area
    
    countries[f"highway_pc_{y}"] = countries[f"highway_{y}"] / (countries[f"population_{y}"] / 1e6)   # PC - per capita
    countries[f"highway_pa_{y}"] = countries[f"highway_{y}"] / (countries.area / 1e6)         # PA - per area

In [ ]:
explore_column(countries, ['highway_pc_2022', 'population_2022'])

In [ ]:
explore_column(countries, ['highway_pa_2022', 'population_2022'])

Dále spočteme nárusty napříč lety abychom zjistili, kterému státu se daří budovat nejvíce.

In [ ]:
for year in range(min_year+1, max_year+1):
    current = year
    previous = current - 1
    
    countries[f'railway_pc_yoy_{current}'] = countries[f'railway_pc_{current}'] - countries[f'railway_pc_{previous}']
    countries[f'railway_pa_yoy_{current}'] = countries[f'railway_pa_{current}'] - countries[f'railway_pa_{previous}']
    countries[f'highway_pc_yoy_{current}'] = countries[f'highway_pc_{current}'] - countries[f'highway_pc_{previous}']
    countries[f'highway_pa_yoy_{current}'] = countries[f'highway_pa_{current}'] - countries[f'highway_pa_{previous}']

Vypočítejme průměrné umístění, abychom získali finální seznam států podle toho, jak se jim daří stavět infrastrukturu.

In [ ]:
# Init
countries['rank_rw_pc'] = 0
countries['rank_rw_pa'] = 0
countries['rank_hw_pc'] = 0
countries['rank_hw_pa'] = 0

year_count = 0
country_count = len(countries.index)

for year in common_years:
    if year == min_year:
        continue
    year_count += 1
    countries['rank_rw_pc'] += countries[f'railway_pc_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)
    countries['rank_rw_pa'] += countries[f'railway_pa_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)
    countries['rank_hw_pc'] += countries[f'highway_pc_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)
    countries['rank_hw_pa'] += countries[f'highway_pa_yoy_{year}'].rank(ascending=False, method='min', na_option='keep').fillna(country_count)

# Calculate average ranking across all years
countries['rank_rw_pc'] /= year_count
countries['rank_rw_pa'] /= year_count
countries['rank_hw_pc'] /= year_count
countries['rank_hw_pa'] /= year_count

# Replace 0s with NaN for countries with no data
countries['rank_rw_pc'] = countries['rank_rw_pc'].replace(0, float('nan'))
countries['rank_rw_pa'] = countries['rank_rw_pa'].replace(0, float('nan'))
countries['rank_hw_pc'] = countries['rank_hw_pc'].replace(0, float('nan'))
countries['rank_hw_pa'] = countries['rank_hw_pa'].replace(0, float('nan'))

A zobrazme v mapě.

In [ ]:
explore_column(countries, ['rank_rw_pc'])

In [ ]:
countries[['CNTR_CODE', 'NAME_LATN', 'rank_rw_pc']].sort_values('rank_rw_pc').head(10)

In [ ]:
countries[['CNTR_CODE', 'NAME_LATN', 'rank_rw_pa']].sort_values('rank_rw_pa').head(10)

In [ ]:
countries[['CNTR_CODE', 'NAME_LATN', 'rank_hw_pc']].sort_values('rank_hw_pc').head(10)

In [ ]:
countries[['CNTR_CODE', 'NAME_LATN', 'rank_hw_pa']].sort_values('rank_hw_pa').head(10)